In [11]:
import numpy as np
import operator
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten
#from keras.datasets import imdb
import pandas as pd
import numpy as np
#устанавливаем seed
np.random.seed(42)
import pymorphy2
import re
import datetime
import sklearn.feature_extraction.text as fex
morph = pymorphy2.MorphAnalyzer()

import tomita
#from tparser import tomita
from tomita_parser import text_parse
import pandas as pd
def normalize(text):
    return ' '.join([morph.parse(str(w))[0].normal_form for w in text.split()])
def clean_text(text):
    text = str(text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-\'>><`', ' ', text) #deleting symbols  
    text = re.sub('-', ' ', text) #deleting symbols  
    #text = ' '.join(word[:] for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")
    return text 


In [146]:
#df_train = pd.read_csv('dataset1.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
#df_trainR = pd.read_csv('datasetNormFinal.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
df_trainR = pd.read_csv('datasetNormFinal_v2.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)



In [147]:
df_train=df_trainR.copy()

In [148]:
category_data = df_train[df_train[catNameColName].str.contains("MARTPHON")==True]
print(type(category_data))
category_data.iloc[2]

<class 'pandas.core.frame.DataFrame'>


Unnamed: 0                                                   160141
Unnamed: 0.1                                                 160141
PRODUCT                                                    30005103
NAME              Смартфон HTC P3700 TOUCH                      ...
CATEGORY_ID                                                 2050101
BRAND_ID                                                       1045
CATEGORY_NAME            SMARTPHONES                               
BRAND_NAME                                     HTC                 
TYPE                                                         REVIEW
SUBJECT                                                     Diamond
TEXT              Пользуюсь даймондом почти год. Претензий нет, ...
POSTED_DATE                                       19.08.09 12:23:40
RATING                                                            5
BENEFITS                                                        NaN
DRAWBACKS                                       

In [149]:
prodIdList = [30025022,30025023,30024754,30026026]

testList = df_train[(df_train[tovarIdColName] == prodIdList[0])]
print(len(testList))

for prId in prodIdList[1:]:
    testList = pd.concat([testList,df_train[(df_train[tovarIdColName] == prId)]])
print(len(testList))
testList.to_csv("newProd2.csv")

44
139


In [150]:
print(len (df_train))
for prId in prodIdList:
    df_train = df_train[(df_train[tovarIdColName] != prId)] 
    #                & (df_train[tovarIdColName] != 30025023 )]
print(len (df_train))

270941
270802


In [154]:
textColName = "TEXT"
negatColName =  "DRAWBACKS"
positColName = "BENEFITS"
ratingColName = "RATING"
catColName = "CATEGORY_ID"
brandColName = "BRAND_ID"
catNameColName= "CATEGORY_NAME"
tovarIdColName = "PRODUCT"
fTextColName = 'fullText'
fTextColNameNorm = 'fullTextNorm'
tomitaColName = 'tomitaCol'
normPosColName = "NormPos"
normNegColName = "NormNeg"
normTextColName = 'NormCpom'
prodNameColName = 'NAME'
#df_train[prodNameColName]

0         ПО для сервиса М.Видео ЦП ПК пакет Office бесс...
1         ПО для сервиса М.Видео ЦП ПК пакет Office на г...
2         ПО для сервиса М.Видео ЦП ПК пакет Kaspersky  ...
3         ПО для сервиса М.Видео ЦП ПК пакет Kaspersky  ...
4         ПО для сервиса М.Видео ЦП ПК пакет Dr.Web Secu...
5         ПО для сервиса М.Видео ЦП ПК пакет Dr.Web Secu...
6         ПО для сервиса М.Видео ПК пакет Office для дом...
7         ПО для сервиса М.Видео ПК пакет Office для дом...
8         ПО для сервиса М.Видео ПК пакет Office 365 пер...
9         ПО для сервиса М.Видео ПК пакет Office 365 пер...
10        ПО для сервиса М.Видео MS Office H&S 2016+Eset...
11        ПО для сервиса М.Видео MS Office H&S 2016+Eset...
12        ПО для сервиса М.Видео Office 365+ESET Nod32 А...
13        ПО для сервиса М.Видео Office 365+ESET Nod32 А...
14        Подписка Xbox . Xbox LIVE 3 месяца            ...
15        Подписка Xbox . Xbox LIVE 3 месяца            ...
16        Подписка Xbox . Xbox LIVE 3 ме

In [152]:
df_test = pd.read_csv('newProd2.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
colList = pd.unique(df_test[catNameColName])
<div id="sample-1" style='display:none;'></div>
<ul>
<li onclick="view('sample-1')"><a href="/#" id="cat_1">1</a></li>
</ul><div id="sample-1" style='display:none;'></div>
<ul>
<li onclick="view('sample-1')"><a href="/#" id="cat_1">1</a></li>
</ul>

     Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1   PRODUCT  \
0        211167        211167          211167  30025022   
1        211168        211168          211168  30025022   
2        211169        211169          211169  30025022   
3        211170        211170          211170  30025022   
4        211171        211171          211171  30025022   
5        211172        211172          211172  30025022   
6        211173        211173          211173  30025022   
7        211174        211174          211174  30025022   
8        211175        211175          211175  30025022   
9        211176        211176          211176  30025022   
10       211177        211177          211177  30025022   
11       211178        211178          211178  30025022   
12       211179        211179          211179  30025022   
13       211180        211180          211180  30025022   
14       211181        211181          211181  30025022   
15       211182        211182          211182  30025022 

In [194]:
maxFeatures = 10
maxPerSub = 4
coefN = float(0.7)
coefS = float(0.5)
from IPython.core.display import display, HTML
        
def printForCat(df_train, df_test,cat):
    category_data = df_train[df_train[catNameColName].str.contains(cat)==True].copy()
    category_data[fTextColName] = category_data[textColName].astype('str') + " "+category_data[positColName].astype('str')
    category_data[fTextColName] = category_data[fTextColName]  +" "+category_data[negatColName].astype('str')

    if (normTextColName in df_train.columns):
       # print ("data already normalized")
        category_data[fTextColNameNorm] = category_data[normTextColName].astype('str') + " "+category_data[normNegColName].astype('str')
        category_data[fTextColNameNorm] = category_data[fTextColNameNorm]  +" "+category_data[normPosColName].astype('str')
    else:
        print("start normalize at" + str (datetime.datetime.now()))
        category_data[fTextColNameNorm] = category_data[fTextColName].apply(clean_text).apply(normalize)
        print("finishaed  collect normalize at " + str (datetime.datetime.now().time()))
    category_data_test = df_test[df_test[catNameColName].str.contains(cat)==True].copy()
    category_data_test[fTextColName] = category_data_test[textColName].astype('str') + " "+category_data_test[positColName].astype('str')
    category_data_test[fTextColName] = category_data_test[fTextColName]  +" "+category_data_test[negatColName].astype('str')
    category_data_test[fTextColNameNorm]=category_data_test[fTextColName].apply(clean_text).apply(normalize)
    textAndProdGrS_test = category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm_test =  category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrS = category_data.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm =  category_data.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
    textAndProdGrS_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
   # print("total category items " + str(len(textAndProdGrSNorm_All)))
    tf_list = np.array(textAndProdGrSNorm_All)
   # print(tf_list.shape)
    id_tlist = textAndProdGrSNorm_test[tovarIdColName].tolist()

    indList = [i for i in range(0,len(tf_list)) if tf_list[i][0] in id_tlist]
    textAndProdGrS_test[tomitaColName] = textAndProdGrS_test[:][fTextColName].apply(text_parse)
    tfidf = fex.TfidfVectorizer()
    #print("start tfidf at" + str (datetime.datetime.now().time()))
    tfIdfMatrix = tfidf.fit_transform(tf_list[:,1])
    #print("end tf Idf at " + str (datetime.datetime.now().time()))
    ftNames = tfidf.get_feature_names()
    densM = tfIdfMatrix.todense()
    for ind in indList[:]:
        print("#"*50)
        display(HTML('<h2>Продукт</h3>'))
        display(HTML(df_test[df_test[tovarIdColName] == tf_list[ind][0]][prodNameColName].iloc[0]))
        col = densM[ind].tolist()[0]
        valD = { ftNames[i]:v for i,v in enumerate(col) if v>0  }
        sorted_topWords = sorted(valD.items(), key=operator.itemgetter(1), reverse=True)
        sorted_topWords = [ s for s in sorted_topWords if len(s[0])>3]
        dict_r = { s[0]:s[1] for s in sorted_topWords}
        tomitaD = textAndProdGrS_test[textAndProdGrS_test[tovarIdColName] == textAndProdGrS_All.iloc[ind][tovarIdColName]][tomitaColName]
        sublist = list([x[::,0] for x in tomitaD.values][0])
        set_ret = list()
        tomitaPairs = tomitaD.tolist()[0]
        #print("start word rating rebuild at " + str (datetime.datetime.now().time()))
        setSent = list()
        for ts in tomitaPairs:
            if ts[1] in setSent:
                continue
            setSent.append(ts[1])

            ret =0
            tsn = normalize(ts[0])
            if tsn in dict_r.keys():
                ret+=dict_r[tsn]*float(coefS)
            retList = list()
            for tsPr in ts[1].split(" "):
                tsPrn = normalize(tsPr)
                if tsPrn in dict_r.keys():
                    retList.append(dict_r[tsPrn])
            retList.sort(reverse = True)
            coef = coefN
            for retN in retList:
                ret+=float(retN)*coef
                coef*=coef
            
            set_ret.append([ts[1],ret,ts[0]])
        #print("end word rating rebuild at " + str (datetime.datetime.now().time()))
        sorted_sent = sorted(set_ret, key=operator.itemgetter(1), reverse=True)
        subDict = dict()
        curInd = 0
        htmlStr = """<table border="0"> <tr> <th>Особенности</th>
    <th>Важность</th>
    </tr> """
        for feature in sorted_sent:
            if  str(feature[2]) in subDict.keys():
                subDict[feature[2]]+=1
            else:
                subDict[feature[2]]=1
            if (subDict[feature[2]] < maxPerSub):
                featText = list(feature[0].lower())
                featText[0] = featText[0].upper()
                featText = "".join(featText)
                #+  featText.Substring(1);
                htmlStr +=" <tr><td> "
                htmlStr += "  "+featText+ " </td><td>"
                htmlStr += "  "+str(int(float(feature[1])*100))+ " </td><tr>"

                #print(" особенность: "+str(featText)+ " важность "+ 
                #      str(int(float(feature[1])*100)))
                      #+ "сущ" + str(feature[2]))
                curInd+=1
            if curInd > maxFeatures:
                break
        display(HTML(htmlStr))
                      

In [195]:
cat = colList[0][1:]
#print(colList)
for cat in colList:
    display(HTML('<h1>'+str(cat) +'</h1>'))
    printForCat(df_train, df_test,cat[1:])

##################################################


Особенности,Важность
Быть телефон с емкой батарея,34
Отличное качество для такой стоимости телефона мощная батарея,34
Батарея телефон слабая,33
Телефон за эти деньги просто,26
Хороший телефон,25
Телефон который у меня был,25
Камера телефон плохая,25
Хотеть приобрести недорогой аппарат с хорошей камерой и мощным заряд батареи,24
Еще уже третий день в настройках не могу найти будильник неплохая батарея камера,23
Камера сильно разочаровала режим hdr вообще лучше,23


##################################################


Особенности,Важность
Хороший телефон,22
Весь прочему в магазине как обычно нет никаких аксессуаров к таким бюджетным телефон,21
Также достаточно хорошая камера,21
Камера очень даже не плохие,20
Батарея очень долго держится,20
Хорошая батарея,20
Хорошая камера,20
Фото день делает хорошие,19
Телефон советовать и цена у него приемлемая и сама,18
Сервер подключаюсь не хочет,18


##################################################


Особенности,Важность
Камера очень очень слабая,31
Камера плохая и видео,30
Камера отличная и качество изображения тоже,29
Фото-видео камера средняя если привыкнуть к автофокусу то оновная камера в дневное время делает хорошие снимки и видео,26
Фото или видео для себя тогда лучше,25
Отличная камера с вспышкой делает потрясающие фотография,23
Хороший современный телефон,23
Видео 5.нужно лишь настроить камеру и не использовать украшение,22
Телефон хороший,22
Хороший телефон,22


##################################################


Особенности,Важность
Этот телефона samsung пожалуйста добавьте в этот телефон,37
Очень довольны телефон,33
Телефон выглядеть очень стильно,32
Экран просто бомба не смотря что телефон не дорогой,26
Прожить телефон 6 день,26
Купить три дня назад j3 все нечего но аккумулятор разряжается за день вроде и те так часто и сижу в интернете а так смартфон,24
Очень обидненько телефон хороший в план работы,24
Экран все открывается и работает шустро,23
Иметь не очень хорошую камера,23
Камера неплохая очень,22
